In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.photUtils as photUtils

In [2]:
import bandpassUtils as bu
import calcM5 as m5

In [3]:
defaultDirs = bu.setDefaultDirs()
addLosses = True
atmosphere = bu.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = bu.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)
atmosphere12 = bu.readAtmosphere(defaultDirs['atmosphere'], atmosFile='pachonModtranAtm_12_aerosol.dat')
hardware2, system2 = bu.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere12)

In [4]:
eff_wavelen = m5.get_effwavelens(system)

In [5]:
%%timeit
for airmass in (1.0, 1.1, 1.2):
    fwhm_eff, fwhm_geom = m5.scale_seeing(0.6, eff_wavelen, airmass=airmass)

37 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%%timeit
airmass = np.array([1.0, 1.1, 1.2])
fwhm_eff, fwhm_geom = m5.scale_seeing(0.6, eff_wavelen, airmass)

24.2 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
airmass = np.array([1.0, 1.1, 1.2])
fwhm_eff, fwhm_geom = m5.scale_seeing(0.62, eff_wavelen, airmass)
print('X=1.0, each filter', np.array2string(fwhm_eff[:,0], precision=2))
defaults = photUtils.LSSTdefaults()
print(defaults.FWHMeff('u'), defaults.FWHMeff('g'), defaults.FWHMeff('r'), defaults.FWHMeff('i'), 
      defaults.FWHMeff('z'), defaults.FWHMeff('y'))
print('g band, all airmasses', fwhm_eff[1])

X=1.0, each filter [ 0.93  0.88  0.83  0.8   0.78  0.76]
0.92 0.87 0.83 0.8 0.78 0.76
g band, all airmasses [ 0.87558412  0.92711475  0.97680219]


In [8]:
t2 = m5.calcM5(hardware, system, atmosphere, X=1.0, return_t2_values=True)

Filter FWHMeff FWHMgeom SkyMag SkyCounts Zp_t Tb Sb kAtm Gamma Cm dCm_infinity m5 SourceCounts
u 0.93 0.81 22.99 70.4 26.92 0.033 0.052 0.4910 0.038161 23.09 0.62 23.77 543.87
g 0.88 0.77 22.26 406.4 28.37 0.125 0.152 0.2134 0.038746 24.42 0.18 24.81 797.38
r 0.83 0.73 21.20 812.6 28.15 0.102 0.115 0.1260 0.038986 24.44 0.10 24.36 985.71
i 0.80 0.71 20.48 1144.0 27.83 0.076 0.083 0.0962 0.039088 24.32 0.07 23.92 1096.42
z 0.78 0.69 19.60 1715.5 27.42 0.052 0.056 0.0687 0.039217 24.16 0.05 23.35 1276.72
y 0.76 0.68 18.61 2283.5 26.64 0.026 0.030 0.1703 0.039298 23.73 0.04 22.45 1424.68


In [9]:
t2_12 = m5.calcM5(hardware2, system2, atmosphere12, X=1.2, return_t2_values=True)

Filter FWHMeff FWHMgeom SkyMag SkyCounts Zp_t Tb Sb kAtm Gamma Cm dCm_infinity m5 SourceCounts
u 1.04 0.90 22.99 70.4 26.82 0.030 0.052 0.5878 0.038348 23.11 0.63 23.56 605.15
g 0.98 0.85 22.26 406.4 28.32 0.120 0.152 0.2557 0.038874 24.43 0.18 24.65 887.94
r 0.92 0.81 21.20 812.6 28.12 0.100 0.115 0.1508 0.039089 24.45 0.10 24.21 1098.13
i 0.89 0.78 20.48 1144.0 27.81 0.075 0.083 0.1119 0.039181 24.33 0.07 23.79 1221.68
z 0.87 0.76 19.60 1715.5 27.41 0.052 0.056 0.0808 0.039297 24.16 0.05 23.22 1422.87
y 0.85 0.75 18.61 2283.5 26.62 0.025 0.030 0.1927 0.039370 23.75 0.04 22.31 1587.80


In [10]:
filterlist = ('u', 'g', 'r', 'i', 'z', 'y')
t2['dm5_X1.2'] = {}
for f in filterlist:
    t2['dm5_X1.2'][f] = t2['m5'][f] - t2_12['m5'][f]
t2['dm5_X1.2']

{'g': 0.15909027139515075,
 'i': 0.13307572698634118,
 'r': 0.14198532064238378,
 'u': 0.2127318506398872,
 'y': 0.14009297787482211,
 'z': 0.12981170091778438}

In [11]:
tau = 60.0/30.0
t2['dcm_60'] = {}
for f in filterlist:
    t2['dcm_60'][f] = t2['dCm_infinity'][f] - 1.25 * np.log10(1 + (np.power(10, 0.8*t2['dCm_infinity'][f]) - 1) / (tau))

In [12]:
t2['dm_fid'] = {}
t2['dm_min'] = {}
t2['m5_fid'] = {'u': 23.9, 'g': 25.0, 'r': 24.7, 'i': 24.0, 'z': 23.3, 'y': 22.1}
t2['m5_min'] = {'u': 23.4, 'g': 24.6, 'r': 24.3, 'i': 23.6, 'z': 22.9, 'y': 21.7}
for f in filterlist:
    t2['dm_fid'][f] = t2['m5'][f] - t2['m5_fid'][f]
    t2['dm_min'][f] = t2['m5'][f] - t2['m5_min'][f]

In [13]:
l = ['skyMag', 'FWHMeff', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dcm_60', 'm5', 'dm5_X1.2',  
     'm5_fid', 'dm_fid', 'm5_min', 'dm_min']

writestring = '  \t\t& $u$\t& $g$\t & $r$\t& $i$\t& $z$\t& $y$\t \\\\'
print(writestring)
for li in l:
    if li == 'dCm_infinity':
        writestring = 'dCm_inf' + '\t\t& '
    else:
        writestring = li + '\t\t& '
    if li == 'gamma' or li == 'kAtm':
        mat = '%.3f\t& '
    else:
        mat = '%.2f\t& '
    for f in filterlist:
        writestring += mat % t2[li][f]
    writestring = writestring[:-2] + ' \\\\'
    print(writestring)

  		& $u$	& $g$	 & $r$	& $i$	& $z$	& $y$	 \\
skyMag		& 22.99	& 22.26	& 21.20	& 20.48	& 19.60	& 18.61	 \\
FWHMeff		& 0.93	& 0.88	& 0.83	& 0.80	& 0.78	& 0.76	 \\
gamma		& 0.038	& 0.039	& 0.039	& 0.039	& 0.039	& 0.039	 \\
kAtm		& 0.491	& 0.213	& 0.126	& 0.096	& 0.069	& 0.170	 \\
Cm		& 23.09	& 24.42	& 24.44	& 24.32	& 24.16	& 23.73	 \\
dCm_inf		& 0.62	& 0.18	& 0.10	& 0.07	& 0.05	& 0.04	 \\
dcm_60		& 0.23	& 0.08	& 0.05	& 0.03	& 0.02	& 0.02	 \\
m5		& 23.77	& 24.81	& 24.36	& 23.92	& 23.35	& 22.45	 \\
dm5_X1.2		& 0.21	& 0.16	& 0.14	& 0.13	& 0.13	& 0.14	 \\
m5_fid		& 23.90	& 25.00	& 24.70	& 24.00	& 23.30	& 22.10	 \\
dm_fid		& -0.13	& -0.19	& -0.34	& -0.08	& 0.05	& 0.35	 \\
m5_min		& 23.40	& 24.60	& 24.30	& 23.60	& 22.90	& 21.70	 \\
dm_min		& 0.37	& 0.21	& 0.06	& 0.32	& 0.45	& 0.75	 \\
